In [12]:
from future.builtins import next
import os
import csv
import re
import logging

import dedupe
from unidecode import unidecode

In [13]:
# ## Logging
logging.getLogger().setLevel(logging.DEBUG)

In [24]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column
    
    try : # python 2/3 string differences
        column = column.decode('utf8')
    except AttributeError:
        pass
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column

def readData(filename, encoding, delimiter, header, keyfield):
    """
    Read in our data from a CSV file and create a dictionary of records, 
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename, encoding=encoding) as f:
        reader = csv.DictReader(f, fieldnames=header, delimiter=delimiter)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row[keyfield])
            data_d[row_id] = dict(clean_row)

    return data_d

In [28]:
# ## Import data
filename = r'D:/Geonames/modifications-2018-02-24.txt'
header = ['geonameid','name','asciiname','alternatenames','latitude','longitude','feature class','feature code','country code','cc2','admin1 code','admin2 code','admin3 code','admin4 code','population','elevation','dem','timezone','modification date']
keyfield = 'geonameid'
geonames = readData(filename, 'utf-8', '\t', header, keyfield)

1598

In [34]:
# ## Training
fields = [
    { 'field':'name', 'type':'String' },
    { 'field':'country code', 'type':'Exact', 'has missing':True }
]

deduper = dedupe.Dedupe(fields)
#deduper.sample(geonames, trainingSize)
training = dedupe.trainingDataDedupe(geonames, 'geonameid')